<a href="https://colab.research.google.com/github/bisil2/AI1_Final_Project/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A51_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5(Inception_v4_%EC%B4%88%EA%B8%B0%ED%95%99%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
use_background = 1
use_agumentation = 1
use_dropout = 1
use_l2 = 0

## 라이브러리 설치 및 Import

In [ ]:
''' 패키지 설치 '''
!pip install torch torchvision

# =============================
# torch : 모델 실행, 학습, 추론에 필수적인 PyTorch 프레임워크
# torchvision : 이미지 처리 관련 도구 제공
# =============================

In [ ]:
'''Google Drive 연동'''
from google.colab import drive
drive.mount('/content/drive')

'''필수 라이브러리 import'''
import pandas as pd
import numpy as np
import shutil
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torchsummary import summary
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F

Mounted at /content/drive


## 데이터 불러오기

In [ ]:
'''Label 데이터 불러오기'''
rawdata = pd.read_csv("/content/drive/MyDrive/data/rawdata.csv")

In [ ]:
'''Image 데이터 불러오기'''
if use_background:
  zipName = 'data'
else:
  zipName = 'raw'

# data 폴더 생성
targetPath = '/content/data/'
os.makedirs(targetPath, exist_ok=True)

# 압축 파일 content로 복사 => content에 있으면 처리속도가 비교적 빠름
rootZip = f'/content/drive/MyDrive/data/{zipName}.zip'
targetZip = f'/content/{zipName}.zip'
shutil.copyfile(rootZip, targetZip)

# zipfile.ZipFile로 압축 파일을 열고, 압축된 모든 파일을 targetPath로 이동(압축 해제)
with zipfile.ZipFile(targetZip, 'r') as zip_ref:
  zip_ref.extractall(targetPath)

## 데이터 Split

In [ ]:
''' Train:Val:Test = 6:2:2 분할'''
# rawdata를 분할. train:(val+test) = 6:4
train, temp = train_test_split(rawdata, test_size=0.4, random_state=1)

# rawdata를 분할. val:test = 5:5
val, test = train_test_split(temp, test_size=0.5, random_state=1)

## 데이터셋 클래스, Transform 생성

In [ ]:
''' DataSet 클래스 정의 '''
# =============================
# 목적 : DataFrame에 저장된 이미지 경로와 라벨 등을 불러오고, 전처리 후 (image, label) 리턴
# 매개변수(?)
#  - dataframe : 이미지 파일 이름/클래스 등이 포함된 변수
#  - rootDir : 이미지들이 저장된 경로
#  - transform : torchvision.transforms를 사용한 이미지 전처리 파이프라인(전처리 묶음? 정도로 이해하면 될 듯)
# =============================
class CustomDataset(Dataset):
  def __init__(self, dataframe, rootDir, transform=None):
    self.dataframe = dataframe
    self.rootDir = rootDir
    self.transform = transform

  # 데이터셋의 총 샘플 수 반환
  def __len__(self):
    return len(self.dataframe)

  # idx번째 데이터 리턴
  def __getitem__(self, idx):
    # DataFrame의 idx번째 행을 row에 저장
    row = self.dataframe.iloc[idx]
    # 이미지 경로 불러오기(row['image']는 파일명이므로, 상위 경로와 더해줌; 파일 이름이 '.JPG'로 된 경우도 있어서 통일)
    imgName = os.path.join(self.rootDir, row['image'].replace('.JPG', '.jpg'))

    # 이미지 파일을 열고, RGB로 변환
    image = Image.open(imgName).convert('RGB')

    # trasform이 정의되어 있다면, 전처리 적용
    if self.transform:
        image = self.transform(image)

    # 라벨 저장
    label = row['class']

    # (전처리된 이미지, 라벨) 리턴
    return image, label

In [ ]:
'''Transform 정의'''
# train, val, test 용도에 따라 전처리를 다르게 적용
if use_agumentation:
  transform = {
      'train': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # 확률적으로 이미지를 좌우 반전 (데이터 증강)
          transforms.RandomHorizontalFlip(),
          # -15 ~ +15도 사이로 랜덤 회전 (데이터 증강)
          transforms.RandomRotation(15),
          # 밝기, 대비, 채도 랜덤 조절 (데이터 증강)
          transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
          # 10 % 비율로 좌우 또는 상화 랜덤 이동 (데이터 증강)
          transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로 사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'val': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'test': transforms.Compose([ # val과 동일/ train과 달리 평가 용도기 때문에 val과 test에는 데이터 증강이 없음.
          transforms.Resize((299, 299)),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ])
  }
else:
  transform = {
      'train': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로 사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'val': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'test': transforms.Compose([ # val과 동일/ train과 달리 평가 용도기 때문에 val과 test에는 데이터 증강이 없음.
          transforms.Resize((299, 299)),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ])
  }

In [ ]:
# Dataset 불러오기
trainDataset = CustomDataset(train, '/content/data', transform=transform['train'])
valDataset = CustomDataset(val, '/content/data', transform=transform['val'])
testDataset = CustomDataset(test, '/content/data', transform=transform['test'])

# DataLoader 불러오기
trainLoader = DataLoader(trainDataset, batch_size=32, shuffle=False)
valLoader = DataLoader(valDataset, batch_size=32, shuffle=False)
testLoader = DataLoader(testDataset, batch_size=32, shuffle=False)

## 학습, 테스트 메소드 정의

In [ ]:
''' 학습 모델 '''
# =============================
# 목적 : 모델 학습
# Early Stopping : val loss가 5번 이상 개선이 안될 경우, 종료
# =============================
def TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=20, patience=5):
    # Early Stopping 관련 변수
    # loss 기준값을 최댓값으로 설정
    best_val_loss = float('inf')
    # early stopping counter 0으로 설정
    counter = 0
    # best model 저장 경로 설정
    best_model_path = f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth"

    # Backbone 동결 상태에서 fc layer만 학습
    for epoch in range(numEpochs):
        # Train
        # 모델을 학습 모드로 설정
        model.train()
        runningLoss = 0.0
        runningCorrects = 0

        # 훈련 데이터 반복 학습
        for inputs, labels in trainLoader:
            # 이미지와 라벨(클래스)
            inputs, labels = inputs.to(device), labels.to(device)

            # 기존 그래디언트 초기화
            optimizer.zero_grad()
            # 모델 forward pass
            outputs = model(inputs)
            # 손실함수 계산 (출력, 정답)
            loss = criterion(outputs, labels)
            # 역전파 학습
            loss.backward()
            # 파라미터 업데이트
            optimizer.step()

            # 예측값 계산
            _, preds = torch.max(outputs, 1)
            # 손실 합산
            runningLoss += loss.item() * inputs.size(0)
            # 정확도 합산
            runningCorrects += torch.sum(preds == labels.data)

        # 반복 횟수 단위로 Train 평균 손실 및 정확도 계산
        epochLoss = runningLoss / len(trainLoader.dataset)
        epochAcc = runningCorrects.double() / len(trainLoader.dataset)
        print(f"Epoch {epoch+1}/{numEpochs} - Train Loss: {epochLoss:.4f} Acc: {epochAcc:.4f}")

        # Validation
        # 모델을 평가 모드로 설정
        model.eval()
        valLoss = 0.0
        valCorrects = 0

        # 그래디언트 비활성화
        with torch.no_grad():
            for inputs, labels in valLoader:
                # 이미지와 라벨(클래스)
                inputs, labels = inputs.to(device), labels.to(device)
                # 모델 forward pass
                outputs = model(inputs)
                # 손실함수 계산 (출력, 정답)
                loss = criterion(outputs, labels)

                # 예측값 계산
                _, preds = torch.max(outputs, 1)
                # 손실 합산
                valLoss += loss.item() * inputs.size(0)
                # 정확도 합산
                valCorrects += torch.sum(preds == labels.data)

        # 반복 횟수 단위로 Validation 평균 손실 및 정확도 계산
        valLoss /= len(valLoader.dataset)
        val_acc = valCorrects.double() / len(valLoader.dataset)
        print(f"Validation Loss: {valLoss:.4f} Acc: {val_acc:.4f}")

        # Scheduler: Validation Loss 기반으로 학습률 조정
        scheduler.step(valLoss)

        # Early Stopping
        # 개선이 되면
        if valLoss < best_val_loss:
            best_val_loss = valLoss
            counter = 0
            torch.save(model, best_model_path)
            print(f"Renewal best model: {best_val_loss:.4f}")
        # 개선이 안되면
        else:
            counter += 1
            print(f"No improvement in {counter}/{patience} epochs")
            if counter >= patience:
                break

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
import torch.nn.functional as F

def TestModel(model, testLoader):
    all_preds = []
    all_probs = []  # AUC 계산용 softmax 확률
    all_labels = []

    testCorrects = 0
    testLoss = 0.0

    criterion = nn.CrossEntropyLoss()
    model.eval()

    with torch.no_grad():
        for inputs, labels in testLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 예측 클래스
            _, preds = torch.max(outputs, 1)
            # softmax 확률 (클래스 1의 확률만)
            probs = F.softmax(outputs, dim=1)[:, 1]

            testLoss += loss.item() * inputs.size(0)
            testCorrects += torch.sum(preds == labels.data)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 평균 계산
    testLoss /= len(testLoader.dataset)
    testAcc = testCorrects.double() / len(testLoader.dataset)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    auc = roc_auc_score(all_labels, all_probs)

    print(f"Test Accuracy  : {testAcc:.4f}")
    print(f"Test Loss      : {testLoss:.4f}")
    print(f"ROC-AUC        : {auc:.4f}")
    print(f"Precision      : {precision:.4f}")
    print(f"Recall         : {recall:.4f}")
    print(f"F1-score       : {f1:.4f}")

    return testAcc.item(), testLoss, auc, precision, recall, f1

https://github.com/weiaicunzai/pytorch-cifar100/blob/master/models/inceptionv4.py

In [ ]:
''' Inception-V4 모델 생성 및 정의 '''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class BasicConv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
    super().__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, bias = False, **kwargs),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.conv(x)
    return x

class Stem(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Sequential(
        BasicConv2d(3,32,3,stride = 2, padding = 0), # 149x149x32
        BasicConv2d(32,32,3,padding = 0), # 147x147x32
        BasicConv2d(32,64,3,padding = 1) # 147x147x64
    )

    self.branch_1_pool = nn.MaxPool2d(3,stride=2,padding=0)
    self.branch_1_conv = BasicConv2d(64,96,3,stride=2,padding=0)
    # 73x73x160

    self.branch_2a = nn.Sequential(
        BasicConv2d(160,64,1,stride = 1, padding = 0),
        BasicConv2d(64,96,3,stride = 1, padding=0)
    )
    self.branch_2b = nn.Sequential(
        BasicConv2d(160,64,1,stride = 1, padding = 0),
        BasicConv2d(64,64,(7,1), stride = 1, padding = (3,0)),
        BasicConv2d(64,64,(1,7), stride = 1, padding = (0,3)),
        BasicConv2d(64,96,3,stride = 1, padding=0)
    )
    # 71x71x192

    self.branch_3_pool = nn.MaxPool2d(3,stride = 2, padding = 0)
    self.branch_3_conv = BasicConv2d(192,192,3,stride = 2,padding = 0)
    # 35x35x384

  def forward(self, x):
    x = self.conv1(x)

    x = [
        self.branch_1_pool(x),
        self.branch_1_conv(x)
    ]
    x = torch.cat(x,1)

    x = [
        self.branch_2a(x),
        self.branch_2b(x)
    ]
    x = torch.cat(x,1)

    x = [
        self.branch_3_pool(x),
        self.branch_3_conv(x)
    ]
    x = torch.cat(x,1)

    return x

class InceptionA(nn.Module):
  def __init__(self):
    super().__init__()

    self.branch_a = nn.Sequential(
        nn.AvgPool2d(3,stride=1,padding=1),
        BasicConv2d(384,96,1)
    ) # 35x35x96
    self.branch_b = BasicConv2d(384,96,1,stride=1,padding=0) # 35x35x96
    self.branch_c = nn.Sequential(
        BasicConv2d(384,64,1,stride=1,padding=0),
        BasicConv2d(64,96,3,stride=1,padding=1)
    ) # 35x35x96
    self.branch_d = nn.Sequential(
        BasicConv2d(384,64,1,stride=1,padding=0),
        BasicConv2d(64,96,3,stride=1,padding=1),
        BasicConv2d(96,96,3,stride=1,padding=1)
    ) # 35x35x96
    # 35x35x384
  def forward(self, x):
    x = [
        self.branch_a(x),
        self.branch_b(x),
        self.branch_c(x),
        self.branch_d(x)
    ]
    x = torch.cat(x,1)

    return x

class ReductionA(nn.Module):
  def __init__(self):
    super().__init__()

    # k = 192, l = 224, m = 256, n = 384
    self.branch_a = nn.MaxPool2d(3,stride=2,padding=0) # 17x17x384
    self.branch_b = BasicConv2d(384,384,3,stride=2,padding=0) # 17x17x384
    self.branch_c = nn.Sequential(
        BasicConv2d(384,192,1),
        BasicConv2d(192,224,3,padding=1),
        BasicConv2d(224,256,3,stride=2,padding=0)
    ) # 17x17x256
    # 17x17x1024
  def forward(self, x):
    x = [
        self.branch_a(x),
        self.branch_b(x),
        self.branch_c(x)
    ]
    x = torch.cat(x,1)

    return x

class InceptionB(nn.Module):
  def __init__(self):
    super().__init__()

    self.branch_a = nn.Sequential(
        nn.AvgPool2d(3,stride=1,padding=1),
        BasicConv2d(1024,128,1)
    ) # 17x17x128
    self.branch_b = BasicConv2d(1024,384,1) # 17x17x384
    self.branch_c = nn.Sequential(
        BasicConv2d(1024,192,1),
        BasicConv2d(192,224,(1,7),padding=(0,3)),
        BasicConv2d(224,256,(7,1),padding=(3,0))
    ) # 17x17x256
    self.branch_d = nn.Sequential(
        BasicConv2d(1024,192,1,stride=1,padding=0),
        BasicConv2d(192,192,(1,7),stride=1,padding=(3,0)),
        BasicConv2d(192,224,(7,1),stride=1,padding=(0,3)),
        BasicConv2d(224,224,(1,7),stride=1,padding=(3,0)),
        BasicConv2d(224,256,(7,1),stride=1,padding=(0,3))
    ) # 17x17x256
  # 17x17x1024
  def forward(self, x):
    x = [
        self.branch_a(x),
        self.branch_b(x),
        self.branch_c(x),
        self.branch_d(x)
    ]
    x = torch.cat(x,1)

    return x

class ReductionB(nn.Module):
  def __init__(self):
    super().__init__()

    self.branch_a = nn.MaxPool2d(3,stride=2,padding=0) # 8x8x1024
    self.branch_b = nn.Sequential(
        BasicConv2d(1024,192,1),
        BasicConv2d(192,192,3,stride=2,padding=0)
    ) # 8x8x192
    self.branch_c = nn.Sequential(
        BasicConv2d(1024,256,1),
        BasicConv2d(256,256,(1,7),padding = (3,0)),
        BasicConv2d(256,320,(7,1),padding = (0,3)),
        BasicConv2d(320,320,3,stride=2,padding=0)
    ) # 8x8x320
  # 8x8x1536
  def forward(self, x):
    x = [
        self.branch_a(x),
        self.branch_b(x),
        self.branch_c(x)
    ]
    x = torch.cat(x,1)

    return x

class InceptionC(nn.Module):
  def __init__(self):
    super().__init__()

    self.branch_a = nn.Sequential(
        nn.AvgPool2d(3,stride=1,padding=1),
        BasicConv2d(1536,256,1)
    ) # 8x8x256
    self.branch_b = BasicConv2d(1536,256,1) # 8x8x256
    self.branch_c = BasicConv2d(1536,384,1) # 8x8x384
    self.branch_c_a = BasicConv2d(384,256,(1,3),padding=(0,1))
    self.branch_c_b = BasicConv2d(384,256,(3,1),padding=(1,0))
    # 8x8x512
    self.branch_d = nn.Sequential(
        BasicConv2d(1536,384,1),
        BasicConv2d(384,448,(1,3),padding=(0,1)),
        BasicConv2d(448,512,(3,1),padding=(1,0))
    ) # 8x8x512
    self.branch_d_a = BasicConv2d(512,256,(3,1),padding=(1,0))
    self.branch_d_b = BasicConv2d(512,256,(1,3),padding=(0,1))
    # 8x8x512
    # 8x8x1536
  def forward(self, x):
    t = self.branch_c(x)
    c = [
        self.branch_c_a(t),
        self.branch_c_b(t)
    ]

    t = self.branch_d(x)
    d = [
        self.branch_d_a(t),
        self.branch_d_b(t)
    ]

    a = self.branch_a(x)
    b = self.branch_b(x)
    c = torch.cat(c,1)
    d = torch.cat(d,1)

    x = [a,b,c,d]
    x = torch.cat(x,1)

    return x

class InceptionV4(nn.Module):
  def __init__(self):
    super().__init__()

    self.stem = Stem()

    self.inceptionA = nn.Sequential(
        InceptionA(),
        InceptionA(),
        InceptionA(),
        InceptionA()
    )

    self.reductionA = ReductionA()

    self.inceptionB = nn.Sequential(
        InceptionB(),
        InceptionB(),
        InceptionB(),
        InceptionB(),
        InceptionB(),
        InceptionB(),
        InceptionB()
    )

    self.reductionB = ReductionB()

    self.inceptionC = nn.Sequential(
        InceptionC(),
        InceptionC(),
        InceptionC()
    )

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.dropout = nn.Dropout(0.2*use_dropout)
    self.fc = nn.Sequential(
        nn.Linear(1536, 256),
        nn.ReLU(),
        nn.Dropout(0.2*use_dropout),
        nn.Linear(256, 2)
    )

    self._initialize_weights()

  def forward(self, x):
    x = self.stem(x)
    x = self.inceptionA(x)
    x = self.reductionA(x)
    x = self.inceptionB(x)
    x = self.reductionB(x)
    x = self.inceptionC(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.dropout(x)
    x = self.fc(x)

    return x

  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

In [ ]:
# check Stem
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.randn((3, 3, 299, 299)).to(device)
model = Stem().to(device)
output_Stem = model(x)
print('Input size:', x.size())
print('Stem output size:', output_Stem.size())

In [ ]:
model = InceptionA().to(device)
output_incA = model(output_Stem)
print('Input size:', output_Stem.size())
print('output size:', output_incA.size())

In [ ]:
model = ReductionA().to(device)
output_resA = model(output_incA)
print('Input size:', output_incA.size())
print('output size:', output_resA.size())

In [ ]:
model = InceptionB().to(device)
output_incB = model(output_resA)
print('Input size:', output_resA.size())
print('output size:', output_incB.size())

In [ ]:
model = ReductionB().to(device)
output_resB = model(output_incB)
print('Input size:', output_incB.size())
print('output size:', output_resB.size())

In [ ]:
model = InceptionC().to(device)
output_incC = model(output_resB)
print('Input size:', output_resB.size())
print('output size:', output_incC.size())

In [ ]:
model = InceptionV4().to(device)
summary(model, (3,299,299), device=device.type)

## 모델 학습 및 테스트

In [ ]:
for i in range(0,2):
  # 모델 불러오기
  model = InceptionV4().to(device)

  # CrossEntropyLoss로 손실 함수 설정
  criterion = nn.CrossEntropyLoss()
  # Adam으로 옵티마이저로 사용 (filter를 통해 require_grad = True, 동결되지 않은 것만 업데이트)
  # weight_decay = 4e-5 : L2 정규화 적용으로 과적합 방지 / 논문 l2 정규화 값 0.00004
  optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=4e-5*use_l2)

  # 학습률 스케줄러 설정
  # 검증 손실이 줄어들지 않으면 learning rate 줄임 (수렴 및 안정화)
  # min 모드 : 손실이 최소화되지 않으면 작동 / factor : 학습률을 x배 줄임 / patience : x번 학습동안 개선되지 않으면 발동 / verbose : 메시지 출력
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

  # Train 모델 실행
  TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=20, patience=5)

  os.rename(f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth", f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model_{i+1}.pth")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 - Train Loss: 0.6539 Acc: 0.6047
Validation Loss: 0.5598 Acc: 0.7074
Renewal best model: 0.5598
Epoch 2/20 - Train Loss: 0.4860 Acc: 0.7726
Validation Loss: 0.5461 Acc: 0.7539
Renewal best model: 0.5461
Epoch 3/20 - Train Loss: 0.4276 Acc: 0.8017
Validation Loss: 0.5074 Acc: 0.7907
Renewal best model: 0.5074
Epoch 4/20 - Train Loss: 0.3734 Acc: 0.8295
Validation Loss: 0.7307 Acc: 0.7616
No improvement in 1/5 epochs
Epoch 5/20 - Train Loss: 0.3535 Acc: 0.8366
Validation Loss: 0.4621 Acc: 0.7771
Renewal best model: 0.4621
Epoch 6/20 - Train Loss: 0.2672 Acc: 0.8747
Validation Loss: 0.4927 Acc: 0.8043
No improvement in 1/5 epochs
Epoch 7/20 - Train Loss: 0.2236 Acc: 0.9089
Validation Loss: 0.6159 Acc: 0.7829
No improvement in 2/5 epochs
Epoch 8/20 - Train Loss: 0.2468 Acc: 0.8915
Validation Loss: 0.5380 Acc: 0.8178
No improvement in 3/5 epochs
Epoch 9/20 - Train Loss: 0.2290 Acc: 0.9025
Validation Loss: 0.6265 Acc: 0.7888
No improvement in 4/5 epochs
Epoch 10/20 - Train Loss: 0

In [ ]:
print(f"{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth")
for i in range(2):
  best_model_path = f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model_{i+1}.pth"
  model = torch.load(best_model_path, weights_only=False)
  model.to(device)

  # 테스트
  result = TestModel(model, testLoader)
  print()

1110_best_model.pth
Test Accuracy  : 0.8779
Test Loss      : 0.3697
ROC-AUC        : 0.9343
Precision      : 0.8805
Recall         : 0.8786
F1-score       : 0.8777

Test Accuracy  : 0.8585
Test Loss      : 0.3948
ROC-AUC        : 0.9257
Precision      : 0.8602
Recall         : 0.8591
F1-score       : 0.8584

